In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib
import requests
from pathlib import Path
from datetime import datetime, timedelta

# Load Model and Scaler
MODEL_PATH = Path('../models/cone_model.keras')
SCALER_PATH = Path('../models/scaler.pkl')

if not MODEL_PATH.exists() or not SCALER_PATH.exists():
    raise FileNotFoundError("Model or Scaler not found. Please run training.ipynb first.")

model = tf.keras.models.load_model(MODEL_PATH, compile=False)
scaler = joblib.load(SCALER_PATH)

print("✓ Model and Scaler loaded successfully.")

✓ Model and Scaler loaded successfully.


In [2]:
def get_nasa_weather_forecast(lat, lon):
    """
    Fetches recent/forecast weather data. 
    For this demo, we'll fetch the last year's average as a proxy for 'expected conditions'.
    """
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)
    
    start_str = start_date.strftime('%Y%m%d')
    end_str = end_date.strftime('%Y%m%d')
    
    url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    params = {
        'parameters': 'PRECTOTCORR,RH2M,WS2M',
        'community': 'AG',
        'longitude': lon,
        'latitude': lat,
        'start': start_str,
        'end': end_str,
        'format': 'JSON'
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        props = data['properties']['parameter']
        precip = [v for v in props['PRECTOTCORR'].values() if v != -999]
        humidity = [v for v in props['RH2M'].values() if v != -999]
        wind = [v for v in props['WS2M'].values() if v != -999]
        
        return {
            'avg_precip': np.mean(precip) if precip else 0,
            'avg_humidity': np.mean(humidity) if humidity else 0,
            'avg_wind': np.mean(wind) if wind else 0
        }
    except Exception as e:
        print(f"Warning: Could not fetch weather ({e}). Using defaults.")
        return {'avg_precip': 2.5, 'avg_humidity': 65.0, 'avg_wind': 3.5}

def calculate_density(lat, lon, surrounding_trees):
    """
    Calculate tree density (trees per km2) within a 100m radius.
    """
    if surrounding_trees is None or len(surrounding_trees) == 0:
        return 0
        
    # Simple distance filter (approximate)
    # 1 deg lat ~ 111km, 1 deg lon ~ 96km (at 30 deg lat)
    lat_diff = 0.001 # ~110m
    lon_diff = 0.001 # ~100m
    
    nearby = surrounding_trees[
        (surrounding_trees['LATITUDE'].between(lat - lat_diff, lat + lat_diff)) &
        (surrounding_trees['LONGITUDE'].between(lon - lon_diff, lon + lon_diff))
    ]
    
    # Area of 100m radius circle = pi * 0.1^2 = 0.0314 km2
    area_km2 = np.pi * (0.1 ** 2)
    return len(nearby) / area_km2

In [ ]:
def predict_cone_of_uncertainty(patient_zero, surrounding_trees=None, duration_months=6):
    """
    Predicts the spread radius.
    
    patient_zero: dict with 'lat', 'lon', 'date'
    surrounding_trees: DataFrame with 'LATITUDE', 'LONGITUDE'
    duration_months: Prediction horizon (default 6 months)
    """
    lat = patient_zero['lat']
    lon = patient_zero['lon']
    
    # 1. Get Environmental Data
    print("Fetching environmental data...")
    weather = get_nasa_weather_forecast(lat, lon)
    print(f"  Precip: {weather['avg_precip']:.2f} mm/day")
    print(f"  Humidity: {weather['avg_humidity']:.1f} %")
    print(f"  Wind: {weather['avg_wind']:.2f} m/s")
    
    # 2. Calculate Density
    density = calculate_density(lat, lon, surrounding_trees)
    print(f"  Local Tree Density: {density:.1f} trees/km2")
    
    # 3. Prepare Features
    # Features: ['log_duration', 'log_density', 'avg_precip', 'avg_humidity', 'avg_wind']
    duration_days = duration_months * 30
    
    features = pd.DataFrame([{
        'log_duration': np.log1p(duration_days),
        'log_density': np.log1p(density),
        'avg_precip': weather['avg_precip'],
        'avg_humidity': weather['avg_humidity'],
        'avg_wind': weather['avg_wind']
    }])
    
    # 4. Scale Features
    features_scaled = scaler.transform(features)
    
    # 5. Predict
    preds = model.predict(features_scaled, verbose=0)[0]
    
    return {
        'Conservative (10%)': max(0, preds[0]),
        'Expected (50%)': max(0, preds[1]),
        'Severe (90%)': max(0, preds[2])
    }

# Example Usage
print("\n--- Running Inference Simulation ---")

# Mock Patient Zero (Austin, TX)
patient_zero = {
    'lat': 30.2672,
    'lon': -97.7431,
    'date': '2024-01-01'
}

# Mock Surrounding Trees (Random nearby points)
surrounding_trees = pd.DataFrame({
    'LATITUDE': np.random.uniform(30.266, 30.268, 50),
    'LONGITUDE': np.random.uniform(-97.744, -97.742, 50)
})

cone = predict_cone_of_uncertainty(patient_zero, surrounding_trees)

print("\nPredicted Cone of Uncertainty (Radius in ft):")
for k, v in cone.items():
    print(f"  {k}: {v:.1f} ft")


--- Running Inference Simulation ---
Fetching environmental data...
  Precip: 2.02 mm/day
  Humidity: 63.5 %
  Wind: 2.68 m/s
  Local Tree Density: 1432.4 trees/km2

Predicted Cone of Uncertainty (Radius in ft):
  Conservative (10%): 65.1 ft
  Expected (50%): 142.4 ft
  Severe (90%): 295.5 ft
